In [1]:
#coding: utf-8
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import datetime
from tqdm import tqdm#_notebook as tqdm
import multiprocessing
import gzip

In [2]:
def getterm(term):
    r = requests.get('https://www.hotels.com/siteindex_www_hotels_com.xml')
    todf = []
    citylinks = [x.text for x in BeautifulSoup(r.content,'lxml').find_all('loc') if '/%s_www' % term in x.text]
    for city in tqdm(citylinks):
        r2 = requests.get(city)
        soup = BeautifulSoup(gzip.decompress(r2.content),'lxml')
        todf.append(pd.DataFrame([{'name':x.find('loc').text.split('/')[-2],
                                   'id':x.find('loc').text.split('/')[-3][2:],
                                   'url':x.find('loc').text
                                  } for x in soup.find_all('url')])
        )
    return pd.concat(todf)

In [3]:
def recmerge(df1,df2,collist,targetcol):
    out = []
    for col in collist:
        if len(out) < 1:
            out.append(df1.merge(df2[[col,targetcol]],on=col,how='left'))
        else:
            out.append(out[-1][out[-1][targetcol].isnull()].drop(targetcol,axis=1
                       ).merge(df2[[col,targetcol]],on=col,how='left'))
    return pd.concat([x[~x['id'].isnull()] for x in out])

In [4]:
fullhotel = getterm('HOTEL')

100%|██████████| 21/21 [01:56<00:00,  4.93s/it]


In [327]:
fullcity = getterm('CITY')

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

In [329]:
fullcity['place'] = fullcity['name'].apply(lambda x: '-'.join(x.split('-')[1:]))
fullcity['place-1'] = fullcity['place'].apply(lambda x: x.split('-')[0])
fullcity['place-2'] = fullcity['place'].apply(lambda x: '-'.join(x.split('-')[:2]))

In [ ]:
c = requests.get('https://www.boldtuesday.com/pages/alphabetical-list-of-all-countries-and-capitals-shown-on-list-of-countries-poster').content
capitals = pd.read_html(c,header=0)[0]
c2 = requests.get('https://en.wikipedia.org/wiki/List_of_largest_cities').content
biguns = pd.read_html(c2,header=0)[1]
c3 = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population').content
bigus = pd.read_html(c3,header=0)[4]
c4 = requests.get('https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits').content
bigeu = pd.read_html(c4,header=0)[0]
citydf = pd.concat([capitals.rename({'COUNTRY':'Nation','CAPITAL':'City'},axis='columns'),
           bigeu.rename({'Country':'Nation'},axis='columns')[['City','Nation']],
           pd.DataFrame({'City':bigus.iloc[:,[1,2]].apply(lambda x: ', '.join(x),axis=1),'Nation':'United States'}),
           biguns[['City','Nation']]]).dropna().apply(lambda x: x.apply(lambda y: y.title()))
citydf['City'] = citydf['City'].apply(lambda x: re.sub('\[.*?\]','',x))
citydf = citydf.drop_duplicates('City')

In [332]:
citydf['place'] = citydf.apply(lambda x: '-'.join(
    x[['City','Nation']]).lower().replace(' ','-'
                                 ).replace(',',''
                                 ).replace('-united-states',''
                                 ).replace("'",'-'
                                 ).replace('á','a')
                               ,axis=1)
citydf = citydf.drop_duplicates('place')
citydf['place-1'] = citydf.apply(lambda x: x['City'].lower(
                                 ).replace(' ','-'
                                 ).replace(',',''
                                 ).replace("'",''
                                 ).replace('á','a')
                               ,axis=1)
citydf['place-2'] = citydf['place-1']

merged_city = recmerge(citydf,fullcity,['place','place-1','place-2'],'id')

In [12]:
def process_hotel(hotelid):
    hotelid = str(hotelid)
    result = {'hotelid':hotelid}
    soup = BeautifulSoup(requests.get('https://www.hotels.com/ho' + hotelid,
                                     params={'locale':'en_IE'}).content,'html5lib')
    for vcard in soup.find_all('div',{'class':'property-description'}):
        for fun, name in [[lambda x: x.find('h1').text,'name'],
                         [lambda x: x.find('span',{'class':'hotel-coordinates'}
                                          ).find_all('meta')[0]['content'],'lat'],
                         [lambda x: x.find('span',{'class':'hotel-coordinates'}
                                          ).find_all('meta')[1]['content'],'lon'],
                         [lambda x: x.find('span',{'class','star-rating-text'}).text,'star'],
                         [lambda x: x.find('span',{'class','street-address'}).text,'street'],
                         [lambda x: x.find('span',{'class','locality'}).text,'locality'],
                         [lambda x: x.find('span',{'class','region'}).text,'region'],
                         [lambda x: x.find('span',{'class','postal-addr'}).text,'addr'],
                         [lambda x: x.find('span',{'class','postal-code'}).text,'zip'],
                         [lambda x: x.find('span',{'class','country-name'}).text,'country'],
                         [lambda x: x.find('div',{'class','tagline'}).text,'tagline']]:
            try:
                result[name] = fun(vcard)
            except:
                #print('FUCKED ----------- %s --- %s' % (name,hotelid))
                pass
    try:
        result['dest-id'] = soup.find('span',{'class':'back-link'}).find('a')['href'].split(
            'destination-id=')[-1].split('&')[-1]
    except:
        #print('FUCKED ----------- %s --- %s' % ('dest-id',hotelid))
        pass
    for findict,name in [[{'data-overview-section-type':"HOTEL_FEATURE"},'base'],
                        [{'data-overview-section-type':"FAMILY_FRIENDLY_SECTION"},'family'],
                        [{'class':'key-facts-container'},'key'],
                        [{'class':'travelling-container'},'travel'],
                        [{'class':'transport-container'},'transport'],
                        [{'class':'badges-and-services-container'},'badges']]:
        baselist = soup.find('div',findict)
        try:
            lis = baselist.find_all('li')
        except:
            #print('FUCKED ----------- %s --- %s' % (name,hotelid))
            lis = []
            result[name + '_missing'] = 1
        for li in lis:
            num_found = False
            for reg,num_name in [['This hotel has (\d+)','rooms'],
                        ['This hotel is arranged over (\d+)','floors'],
                        ['(\d+) restaurants','restaurants'],
                        ['(\d+) outdoor pools','outdoor pools'],
                        ['(\d+) spa tubs','spa tubs'],
                        ['(\d+) bars/lounges','bars/lounges'],
                        ['(\d+) poolside bars','poolside bars'],
                        ['(\d+) poolside bars','poolside bars'],
                        ['(\d) smoke-free guestrooms','smoke-free guestrooms'],
                        ['(\d) guestrooms','guestrooms'],
                        ['(\d) apartments','apartments'],
                        ['(\d) smoke-free apartments','smoke-free apartments'],
                        ['(\d) inch flat-screen TV','inch flat-screen TV'],
                        ['(\d) inch LCD TV','inch LCD TV'],
                        ['(\d) inch TV','inch TV'],
                        ['(\d) Smart TV','Smart TV'],
                        ['(\d) LED TV','LED TV'],
                        ['(\d) inch plasma TV','inch plasma TV'],
                        ['(\d) villas','villas'],
                        ['(\d) smoke-free accommodations','smoke-free accommodations']]:
                ref = re.findall(reg,li.text)
                if len(ref) > 0:
                    result[num_name] = ref[0]
                    num_found = True
            if num_found:
                continue
            if name == 'key':
                if 'Check-in time' in li.text:
                    result['Check-in time'] = li.text.split('time ')[-1]
                    continue
                if 'Check-out time' in li.text:
                    result['Check-out time'] = li.text.split('time ')[-1]
                    continue
            result[name + '_' + li.text] = 1


    for findict,name in [[{'class':'fact-sheets in-the-property-module'},'inhotel'],
                         [{'class':'fact-sheets in-the-room-module'},'inroom']]:    
        inhotel = soup.find('div',findict)
        try:
            lis = inhotel.find_all('div',{'class':'fact-sheet-table-row'})
        except:
            #print('FUCKED ----------- %s --- %s' % (name,hotelid))
            result[name + '_missing'] = 1
            lis = []
        for li in lis:
            head = li.find('div',{'class':'fact-sheet-table-header'})
            try:
                head = head.text
                cells = li.find_all('li')
            except:
                #print('FUCKED ----------- %s --- %s' % (name,hotelid))
                cells = []
            for item in cells:
                result[name + '_' + head + '_' + item.text] = 1
    return result

#@ray.remote
#def ray_hotel_list(hlist):
#    return pd.DataFrame([process_hotel(h) for h in hlist])

def mp_hotel_list(hlist):
    return pd.DataFrame([process_hotel(h) for h in tqdm(hlist)])

In [13]:
splitnum = multiprocessing.cpu_count() * 4
#hotel_atts = pd.concat(ray.get([ray_hotel_list.remote(hlist) for
#                        hlist in np.array_split(
#                        fullhotel['id'].sample(100).values,splitnum)])).set_index('hotelid')

pool = multiprocessing.Pool(splitnum)

hotel_atts = pd.concat(pool.map(mp_hotel_list,[hlist for
                        hlist in np.array_split(
                        fullhotel['id'].sample(100).values,splitnum)])).set_index('hotelid')

100%|██████████| 6/6 [00:27<00:00,  4.56s/it]
/home/borza/Dropbox/Papers_write/ma_thesis/hierarchical-genmodel/pyenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [20]:
hotel_atts.to_csv('../data/external/hotel-atts.csv',index='hotelid')

In [8]:
hotel_atts['country'].value_counts()

United States of America    14
Italy                       10
Spain                        6
China                        5
Australia                    4
Germany                      3
Thailand                     3
United Kingdom               3
Canada                       3
Greece                       3
Indonesia                    3
Turkey                       2
South Korea                  2
Philippines                  2
New Zealand                  2
Mexico                       2
South Africa                 2
France                       2
Nigeria                      1
Hungary                      1
Peru                         1
Uzbekistan                   1
Taiwan                       1
Costa Rica                   1
Dominican Republic           1
Poland                       1
Uruguay                      1
Guatemala                    1
Serbia                       1
Finland                      1
Japan                        1
Kenya                        1
Nepal   

In [464]:
hdf = []
for h in tqdm(fullhotel['id'].sample(10000).values):
    hdf.append(process_hotel(h))
testdf2 = pd.DataFrame(hdf)

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))

FUCKED ----------- star --- 956423680
FUCKED ----------- region --- 956423680
FUCKED ----------- region --- 692070
FUCKED ----------- zip --- 692070
FUCKED ----------- family --- 692070
FUCKED ----------- inhotel --- 692070
FUCKED ----------- family --- 739732128
FUCKED ----------- region --- 856353280
FUCKED ----------- region --- 652368
FUCKED ----------- transport --- 866263872
FUCKED ----------- region --- 744751648
FUCKED ----------- transport --- 778624544
FUCKED ----------- transport --- 395441
FUCKED ----------- star --- 643611
FUCKED ----------- region --- 647578080
FUCKED ----------- zip --- 647578080
FUCKED ----------- star --- 771439968
FUCKED ----------- region --- 771439968
FUCKED ----------- star --- 729957696
FUCKED ----------- region --- 729957696
FUCKED ----------- zip --- 729957696
FUCKED ----------- region --- 778913216
FUCKED ----------- zip --- 778913216
FUCKED ----------- region --- 470616
FUCKED ----------- zip --- 470616
FUCKED ----------- inhotel --- 608970112

FUCKED ----------- transport --- 478837
FUCKED ----------- star --- 689785024
FUCKED ----------- region --- 762569920
FUCKED ----------- star --- 622182880
FUCKED ----------- region --- 622182880
FUCKED ----------- transport --- 616896
FUCKED ----------- star --- 650237
FUCKED ----------- region --- 650237
FUCKED ----------- family --- 854651264
FUCKED ----------- star --- 720868960
FUCKED ----------- region --- 720868960
FUCKED ----------- region --- 728848256
FUCKED ----------- region --- 353782
FUCKED ----------- star --- 799208352
FUCKED ----------- region --- 799208352
FUCKED ----------- region --- 636690
FUCKED ----------- zip --- 636690
FUCKED ----------- region --- 647098464
FUCKED ----------- star --- 272947264
FUCKED ----------- family --- 272947264
FUCKED ----------- travel --- 272947264
FUCKED ----------- inroom --- 272947264
FUCKED ----------- region --- 579884800
FUCKED ----------- family --- 646023520
FUCKED ----------- travel --- 646023520
FUCKED ----------- transport -

FUCKED ----------- region --- 687154976
FUCKED ----------- region --- 581700
FUCKED ----------- region --- 672728
FUCKED ----------- family --- 277107
FUCKED ----------- star --- 770222816
FUCKED ----------- base --- 770222816
FUCKED ----------- family --- 770222816
FUCKED ----------- travel --- 770222816
FUCKED ----------- transport --- 770222816
FUCKED ----------- inhotel --- 770222816
FUCKED ----------- inroom --- 770222816
FUCKED ----------- family --- 771125280
FUCKED ----------- transport --- 683931
FUCKED ----------- star --- 548737
FUCKED ----------- region --- 548737
FUCKED ----------- zip --- 548737
FUCKED ----------- region --- 238287
FUCKED ----------- region --- 759275264
FUCKED ----------- region --- 389127
FUCKED ----------- zip --- 389127
FUCKED ----------- zip --- 523206
FUCKED ----------- region --- 683706304
FUCKED ----------- region --- 883755360
FUCKED ----------- star --- 570252
FUCKED ----------- region --- 570252
FUCKED ----------- star --- 561183
FUCKED -------

FUCKED ----------- region --- 870589120
FUCKED ----------- star --- 95041248
FUCKED ----------- inhotel --- 653698
FUCKED ----------- family --- 956697920
FUCKED ----------- inroom --- 881652512
FUCKED ----------- star --- 952706656
FUCKED ----------- region --- 952706656
FUCKED ----------- zip --- 952706656
FUCKED ----------- region --- 619773
FUCKED ----------- star --- 938652224
FUCKED ----------- base --- 938652224
FUCKED ----------- family --- 938652224
FUCKED ----------- travel --- 938652224
FUCKED ----------- transport --- 938652224
FUCKED ----------- inhotel --- 938652224
FUCKED ----------- inroom --- 938652224
FUCKED ----------- region --- 640245024
FUCKED ----------- region --- 642113
FUCKED ----------- zip --- 642113
FUCKED ----------- star --- 806848832
FUCKED ----------- region --- 806848832
FUCKED ----------- base --- 726353824
FUCKED ----------- family --- 726353824
FUCKED ----------- inhotel --- 726353824
FUCKED ----------- star --- 573971
FUCKED ----------- zip --- 573

...

FUCKED ----------- star --- 956882784
FUCKED ----------- zip --- 523073
FUCKED ----------- inroom --- 523073
FUCKED ----------- star --- 950121440
FUCKED ----------- region --- 750399680
FUCKED ----------- zip --- 750399680
FUCKED ----------- family --- 228558
FUCKED ----------- travel --- 864669696
FUCKED ----------- region --- 737368000
FUCKED ----------- region --- 355762
FUCKED ----------- region --- 613114
FUCKED ----------- zip --- 613114
FUCKED ----------- region --- 686814
FUCKED ----------- star --- 731266848
FUCKED ----------- region --- 731266848
FUCKED ----------- inhotel --- 625244384
FUCKED ----------- star --- 960010656
FUCKED ----------- family --- 960010656
FUCKED ----------- inroom --- 960010656
FUCKED ----------- family --- 396844
FUCKED ----------- travel --- 876870496
FUCKED ----------- transport --- 876870496
FUCKED ----------- region --- 439665
FUCKED ----------- zip --- 639147040
FUCKED ----------- inhotel --- 639147040
FUCKED ----------- star --- 961932768
FUCK

FUCKED ----------- star --- 766079968
FUCKED ----------- region --- 766079968
FUCKED ----------- inhotel --- 766079968
FUCKED ----------- zip --- 854368608
FUCKED ----------- transport --- 668357
FUCKED ----------- inhotel --- 668357
FUCKED ----------- base --- 656711
FUCKED ----------- inhotel --- 656711
FUCKED ----------- star --- 978644864
FUCKED ----------- zip --- 978644864
FUCKED ----------- base --- 978644864
FUCKED ----------- family --- 978644864
FUCKED ----------- travel --- 978644864
FUCKED ----------- transport --- 978644864
FUCKED ----------- inhotel --- 978644864
FUCKED ----------- inroom --- 978644864
FUCKED ----------- zip --- 517393
FUCKED ----------- family --- 517393
FUCKED ----------- inroom --- 517393
FUCKED ----------- star --- 778618080
FUCKED ----------- region --- 778618080
FUCKED ----------- zip --- 778618080
FUCKED ----------- family --- 778618080
FUCKED ----------- inhotel --- 778618080
FUCKED ----------- region --- 338815
FUCKED ----------- region --- 86857

FUCKED ----------- region --- 702261
FUCKED ----------- star --- 746433984
FUCKED ----------- family --- 746433984
FUCKED ----------- inroom --- 746433984
FUCKED ----------- star --- 805032288
FUCKED ----------- region --- 805032288
FUCKED ----------- star --- 684269920
FUCKED ----------- region --- 684269920
FUCKED ----------- base --- 684269920
FUCKED ----------- inhotel --- 684269920
FUCKED ----------- family --- 130325
FUCKED ----------- star --- 878541088
FUCKED ----------- region --- 878541088
FUCKED ----------- region --- 494152
FUCKED ----------- region --- 441171
FUCKED ----------- region --- 972846400
FUCKED ----------- family --- 930619872
FUCKED ----------- region --- 847430272
FUCKED ----------- region --- 728131136
FUCKED ----------- star --- 960277664
FUCKED ----------- inhotel --- 960277664
FUCKED ----------- star --- 482908
FUCKED ----------- zip --- 441583
FUCKED ----------- region --- 411515
FUCKED ----------- region --- 312106
FUCKED ----------- star --- 885500992
F

FUCKED ----------- star --- 894689504
FUCKED ----------- transport --- 649791
FUCKED ----------- inhotel --- 649791
FUCKED ----------- region --- 408202
FUCKED ----------- family --- 253499
FUCKED ----------- transport --- 253499
FUCKED ----------- inroom --- 253499
FUCKED ----------- region --- 543087
FUCKED ----------- region --- 558966
FUCKED ----------- region --- 684478
FUCKED ----------- transport --- 684478
FUCKED ----------- star --- 492578
FUCKED ----------- family --- 937754816
FUCKED ----------- zip --- 562307
FUCKED ----------- family --- 848943808
FUCKED ----------- star --- 435282
FUCKED ----------- family --- 928075840
FUCKED ----------- region --- 826509216
FUCKED ----------- region --- 645554
FUCKED ----------- transport --- 595394
FUCKED ----------- region --- 856031904
FUCKED ----------- inhotel --- 632585
FUCKED ----------- zip --- 623531
FUCKED ----------- star --- 648231776
FUCKED ----------- base --- 648231776
FUCKED ----------- inhotel --- 648231776
FUCKED -----

FUCKED ----------- star --- 536867
FUCKED ----------- star --- 856907584
FUCKED ----------- region --- 856907584
FUCKED ----------- base --- 856907584
FUCKED ----------- inhotel --- 856907584
FUCKED ----------- zip --- 652384
FUCKED ----------- family --- 400419
FUCKED ----------- transport --- 400419
FUCKED ----------- region --- 624428
FUCKED ----------- region --- 742534080
FUCKED ----------- region --- 254331
FUCKED ----------- transport --- 254331
FUCKED ----------- zip --- 621201760
FUCKED ----------- base --- 621201760
FUCKED ----------- inhotel --- 621201760
FUCKED ----------- star --- 644173056
FUCKED ----------- region --- 644173056
FUCKED ----------- region --- 489071
FUCKED ----------- transport --- 364328
FUCKED ----------- family --- 114462
FUCKED ----------- star --- 593793
FUCKED ----------- region --- 593793
FUCKED ----------- region --- 646357504
FUCKED ----------- inhotel --- 603198880
FUCKED ----------- region --- 649222080
FUCKED ----------- star --- 416281
FUCKED 

In [469]:
testdf2.to_csv('../data/external/hotel-atts.csv')

In [370]:
hotelcols = testdf.columns

,0,1,2
86,base_,outdoor pools and,spa tubs
89,base_,restaurants and,bars/lounges
90,base_,restaurants and,poolside bars
1294,transport_Area shuttle within,.,km *
1320,transport_Free area shuttle within,.,mi
1356,travel_,per room (up to,kg)
1361,travel_,per room (up to,lb)
1423,travel_Free children's club (ages,-,)
1458,travel_Up to,children (,year old and younger) stay free when occupying...
1459,travel_Up to,children (,years old and younger) stay free when occupyin...


In [461]:
doubles = pd.Series(hotelcols).apply(lambda x: re.findall('(.*?)\d+(.*)',x))
doubles = doubles[doubles.str.len() > 0]
doubles = doubles.apply(lambda x: pd.Series(x[0]).apply(lambda x: x.strip()))#.drop_duplicates()
doubles.apply(lambda x: '-'.join(x),axis=1).value_counts().reset_index()['index']

0                            base_-smoke-free guestrooms
1                                       base_-guestrooms
2                                       base_-apartments
3                            base_-smoke-free apartments
4            inroom_Be entertained_--inch flat-screen TV
5      travel_Children (-years old and younger) stay ...
6      travel_One child (-years old and younger) stay...
7                    inroom_Be entertained_--inch LCD TV
8                           key_Minimum check-in age is-
9      travel_Children (-years old and younger) not a...
10                       inroom_Be entertained_--inch TV
11                   inhotel_Food and drink_-restaurants
12                 inroom_Be entertained_--inch Smart TV
13                   inroom_Be entertained_--inch LED TV
14                                     base_-restaurants
15                        key_Check-in time starts at-PM
16                             key_Check-in time-PM-9 PM
17                             

In [420]:
testdf[~testdf['key_Monday - Sunday: 8 AM - 9 PM'].isnull()]['hotelid']

60    428319
Name: hotelid, dtype: object

In [352]:
rdate = requests.get('https://www.hotels.com/ho281147/?q-check-out=2018-10-17&tab=description&q-room-0-adults=2&YGF=3&q-check-in=2018-10-16&MGT=1&WOE=3&WOD=2&ZSX=0&SYE=3&q-room-0-children=0&locale=en_IE')

In [353]:
print(BeautifulSoup(rdate.content,'html5lib').find('div',{'class':'room-details resp-module'}).parent.parent.parent.prettify())#.find('form').find_all('input')

AttributeError: 'NoneType' object has no attribute 'parent'

In [148]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:64108 to respond...
Waiting for redis server at 127.0.0.1:40889 to respond...
Starting local scheduler with the following resources: {'CPU': 4, 'GPU': 1}.

View the web UI at http://localhost:8889/notebooks/ray_ui65404.ipynb?token=635ddf85a5b937b7acf9ea3a76974edaa1e27c2705cb447f



{'local_scheduler_socket_names': ['/tmp/scheduler77783717'],
 'node_ip_address': '146.110.60.131',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store38104503', manager_name='/tmp/plasma_manager65169403', manager_port=61224)],
 'raylet_socket_names': [],
 'redis_address': '146.110.60.131:64108',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui65404.ipynb?token=635ddf85a5b937b7acf9ea3a76974edaa1e27c2705cb447f'}

In [421]:
browser = webdriver.Firefox()

ValueError: cannot reindex from a duplicate axis

,Nation,q,url
City,,,
Kabul,Afghanistan,NaN,NaN
Tirana,Albania,NaN,NaN
Algiers,Algeria,"Algiers, Algeria",1635760
Andorra La Vella,Andorra,"Andorra la Vella, Andorra",1634148
Luanda,Angola,NaN,NaN
Saint John'S,Antigua & Barbuda,"St. John's, Antigua (ANU-V.C. Bird Intl.)",1894
Buenos Aires,Argentina,"Buenos Aires, Argentina",14534
Yerevan,Armenia,NaN,NaN
Canberra,Australia,"Canberra, Australian Capital Territory, Australia",1633564


[array(['Kabul', 'Tirana', 'Luanda', 'Yerevan', 'Vienna', 'Manama',
        'Dhaka', 'Minsk', 'Belmopan', 'Thimphu', 'Gaborone', 'Ouagadougou',
        'Praia', 'Ottawa', 'Kinshasa', 'Zagreb', 'Havana', 'Nicosia',
        'Copenhagen', 'Djibouti (City)', 'Roseau', 'Quito', 'Tallinn',
        'Palikir', 'Helsinki', 'Paris', 'Libreville', 'Tbilisi',
        'Guatemala City', 'Conakry', 'Georgetown', 'Reykjavik', 'Jakarta',
        'Tehran', 'Baghdad', 'Dublin', 'Jerusalem, Tel Aviv', 'Rome',
        'Kingston', 'Tokyo', 'South Tarawa', 'Pristina', 'Kuwait City',
        'Vientiane', 'Riga', 'Maseru', 'Monrovia', 'Tripoli', 'Vaduz',
        'Vilnius'], dtype=object),
 array(['Luxembourg', 'Skopje', 'Lilongwe', 'Kuala Lumpur', 'Male',
        'Valletta', 'Majuro', 'Nouakchott', 'Port Louis', 'Mexico City',
        'Chisinau', 'Monaco', 'Ulaanbaatar', 'Podgorica', 'Rabat',
        'Maputo', 'Nay Pyi Taw', 'Yaren District', 'Kathmandu',
        'Wellington', 'Managua', 'Pyongyang', 'Oslo', '

In [ ]:
HEADERS = {'Accept-Language':'en-US,en;q=0.5',
           'Cookie':'SSLB=1; SSID=CABKoh3gAFAAAAAod3dZgmrACCh3d1kLAAAAAADhxdVatXboWQCSjOoMAAWdiQAAtXboWQEAGQ0ABzaLAABzJK1ZCgANDQAFi4oAALV26FkBACgNAAf3iwAAtXboWQEAGg0AAzmLAABzJK1ZCgDhDAAF54gAALV26FkBAMQMAAVfhwAAtXboWQEA5AsABXB8AAC1duhZAQC1DAAF5YUAALV26FkBAC4NAAVxjAAAtXboWQEALQ0ABW6MAAC1duhZAQAuDAAFFIEAALV26FkBAM0MAAWbhwAAtXboWQEAAw0AB0mKAABzJK1ZCgD9DAAFEYoAALV26FkBAAwNAAGFigAAtXboWQEAPAwAANIMAAAeDAAAwwwAAKsKAABXDAAAigwAABENAAC7DAAAwQwAALgMAAAjDQAAuQwAANcKAAAkDQAAOgwAAO8MAAAiDQAA8wsAAAQKAAA; SSRT=h4DoWQIDAQ; SSPV=fFoAAAAAACABBgAAAAAAAAAAAAQAAAAAAAA; user=QSplbl9JRXxIQ09NX0VNRUE.; mvthistory=eJxNlUuuHDEIRXdUMj9jPIuiSJk8ZQkZZw9ZfHBxcWfUxxjzpzrWI894%2FjBvkpXEm%2Bd46Mh0s3vC2DLGM1%2B1ZLa6ti0yC3mLn%2FsjnVvDHj5MtG2M14JZvnoVfHsM3Mder9Ox14z69dKPoWWabIfaK6MxAkDaAFeRPFdHQzS9wqFUosB7DoIGnwOXCtM5CPTXOejnRgZcKV2AOWV5IuBdBbHpVEiPbxtVwYRVkcph6BpVtuQodELVgxbDy3Kqh354ljC0gMeKNstEVEJy9CWzYVrdjVRhKs%2FMs7JhYa7rbPztpxy%2B1WRRvJI5AEtgdPPJhjEvltaogmVDS9mWAUIbEP4cXPOXbeYp3fNzY4JD8kRizuszmgzz7nIj9TbsUQXlhQHkJYgiUH0OzFq2mBsUYN5VkZF5OMZBxt2ItQCx7j6QSAnP4twlGeU4IQD0eaKCMcz25AHvFcGKooiiTl2PZNwae4WW5RMzZOGCGFd1%2B32zGKajenhqmHySyNvotKIWYesYZU0J454QAOxsLoJ255Xm7ZeSj54jpWg7aI0yOqKsDrCW2JXABU%2Fv5qoMbMI8DLNZoY%2FCbOHqYqX07GsNDZ3D%2FP8A4%2FqqBYKOPCgiUkP22raNZhc12f%2FjVSyH4yNnugGacM9vst%2BCmcSHleBIsQ7zMIpnzre05gph13jia6OTUCpLZu1PRLL1XOucaLWP1YAOu9iN2adeXhgTDWnowAI7qrHKbH75F4DxH8BVw78%2Fv%2F%2F6%2Bv3j68e3fyveV0M%3D; guid=8577897d-e0dd-41b6-bb87-e0c7623e1ac6; s_fid=46E32F2FDCBD2CD8-26FE0E1C4D45ACA6; _ga=GA1.2.725440213.1501001517; s_vi=[CS]v1|2CBBBB9685316B18-4000010B400016D2[CE]; AMCV_C00802BE5330A8350A490D4C%40AdobeOrg=-1330315163%7CMCAID%7C2CBBBB9685316B18-4000010B400016D2%7CMCIDTS%7C17459%7CMCMID%7C58816101718994981334228947833632971511%7CMCAAMLH-1509011767%7C6%7CMCAAMB-1509011767%7CTGF0eZrks1DsVMCPfHyQDFnTcTvl83liCN-3ux4fEx6Rm-E%7CMCOPTOUT-1508414167s%7CNONE; 13211=2CBBBB9685316B18-4000010B400016D2; _cc=AdIP8KWCSu4DZRs5hysg6HJa; homepage_search_data=Vmllbm5hLCBBdXN0cmlh%2F%2F15%2F01%2F2018%2F%2F16%2F01%2F2018%2F%2F2%2F%2Fdd%2FMM%2Fyyyy%2F%2F39254%2F%2F; _vz=viz_59ad248236eca; akacd_pr_5=1510826165~rv=74~id=e4754031f85979d8cb7c31e0bfc6ef8b; _gid=GA1.2.1096243265.1508406967; searchReturningUser=true; propertyReturningUser=true'}

APIURL = 'https://www.hotels.com/search/listings.json'

def get_dict(destid,chout,chin):
    i = 1
    out = []
    while True:
        print i
        params = {'destination-id':destid,
                  'q-check-out=':chout,
                  'q-check-in':chin,
                  'q-room-0-adults':2,
                  'q-rooms':1,
                  'q-room-0-children':0,
                  'locale':'en_IE',
                  'pn':i}
        page = requests.get(APIURL,params=params,headers=HEADERS)
        datastring = page.content
        #datastring = script.text.strip().replace('&amp;','&').split(';')[0][4:]
        datastring = datastring.replace('\t','')
        datastring = datastring.replace('true','True')
        datastring = datastring.replace('false','False')
        exec('json = ' + datastring)
        #DO THE D'S
        pno = json['data']['body']['searchResults']['pagination']['currentPage']
        if pno < i:
            break
        i += 1
        res = json['data']['body']['searchResults']['results']
        dlist = make_list_of_dicts(res)
        out += copy.deepcopy(dlist)
    return out


def make_list_of_dicts(res):
    out = []
    for d in res:
        next_d = {}
        for key in d:
            if type(d[key]) == dict:
                for key2 in d[key]:
                    if type(d[key][key2]) == dict:
                        for key3 in d[key][key2]:
                            next_d[key + '-' + key2 + '-' + key3] = d[key][key2][key3]
                    else:
                        next_d[key + '-' + key2] = d[key][key2]
            elif type(d[key]) == list:
                j = 0
                for elem in d[key]:
                    j += 1
                    if type(elem) == dict:
                        for key2 in elem:
                            #print key + '-' + str(j) + '-' + key2
                            #print elem[key2]
                            next_d[key + '-' + str(j) + '-' + key2] = elem[key2]
                    else:
                        next_d[key + '-' + str(j)] = elem
            else:
                next_d[key] = d[key]
        out.append(copy.deepcopy(next_d))
    return out
    

l = len(IDS)

full = []

l = 2

for k in range(l):
    print k, ' OF ', l
    for dates in DATES[:2]
        print dates
        dlist = get_dict(IDS[k],dates[0],dates[1])
        full += copy.deepcopy(dlist)

df = pd.DataFrame(full)

df.to_csv('out.csv')



